In [12]:
import numpy as np
from numba import jit, njit, vectorize, cfunc

In [13]:
N=10000000
#N=1000000
#N=100000
#N=10000
#N=1000
#N=100

In [14]:
a=np.random.rand(N)
b=np.random.rand(N)


Re=4000*np.exp(np.log(1e8/4000)*np.random.rand(N))
epsD=1e-6*np.exp(np.log(0.05/1e-6)*np.random.rand(N))


In [15]:
@njit(parallel=True)
def f_sum(Re,epsD):
    return Re+epsD

@njit(parallel=True)
def f_mult(Re,epsD):
    return Re*epsD

@jit(parallel=True)
def f_mult6(Re,epsD):
    for i in range(6):
        Re*epsD
    return Re*epsD

@jit(parallel=True)
def f_log(Re,epsD):  
    return np.log(Re)

#@jit(parallel=True)
def f_where(Re,epsD):  
    z=np.where(Re>1e5)*1+np.where(Re<1e5)*2

In [16]:
NN=16
name_=['' for i in range(NN+1)]
time_=np.zeros(NN+1)
MARE_=np.zeros(NN+1)

In [17]:
name_[0]='swamee jain'
@njit(parallel=True)
def f_0(Re,epsD): 
    z=np.log(epsD/3.7+5.74/np.exp(0.9*np.log(Re)))
    return 1.3254745276195998/(z*z) #0.25*np.power(np.log(10),2)

name_[1]='cw30'
@njit(parallel=True)
def f_1(Re,epsD):
    z=-8.6*np.ones(len(a))
    y1=epsD/3.7
    y2=-2.1801582991543236/Re   #(-2/np.log(10)*2.51)
    #manual unrolling not done
    for i in range(6):
        z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    return 1.3254745276195998/(z*z) #1/np.power((-2/np.log(10)),2)

name_[2]='cw5'
@njit(parallel=True)
def f_2(Re,epsD):
    z=-8.6*np.ones(len(Re))
    y1=epsD/3.7
    y2=-2.1801582991543236/Re   #(-2/np.log(10)*2.51)
    z=np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*np.log(y1+y2*z)))))
    return 1.3254745276195998/(z*z) #1/np.power((-2/np.log(10)),2)


name_[3]='goudar-sonnad'
@jit(parallel=True)
def f_3(Re,epsD): #'Goudar–Sonnad'
    #
    a_=2/np.log(10)
    b_=epsD/3.7
    d_=np.log(10)*Re/5.02
    s_=b_*d_+np.log(d_)
    q_=np.exp(np.log(s_)*s_/(s_+1))
    #q_=np.power(s_,s_/(s_+1))
    g_=b_*d_+np.log(d_/q_)
    z_=np.log(q_/g_)
    dla_=z_*g_/(g_+1)
    dcfa_=dla_*(1+(z_/2)/((g_+1)*(g_+1)+z_/3*(2*g_-1)))
    f=a_*(np.log(d_/q_)+dcfa_)
    f=1/(f*f)
    #
    return f

name_[4]='haaland_1983'
@jit(parallel=True)
def f_4(Re,epsD): 
    f=np.log10(epsD/3.7+np.exp(np.log(7/Re)*0.9))
    f=0.25/(f*f)
    #
    return f

name_[5]='Barr1981'
@jit(parallel=True)
def f_5(Re,epsD): 
    y1=-2*np.log10(epsD/3.7+4.518*np.log10(Re/7)/(Re*(1+np.exp(np.log(Re)*0.52)/29*np.exp(np.log(epsD)*0.7))))
    f=1/(y1*y1)
    #
    return f

name_[6]='serghides1984'
@jit(parallel=True)
def f_6(Re,epsD): 
    y1=epsD/3.7
    y2=-0.8685889638065035*np.log(y1+12/Re)   #2/np.log(10)
    y3=-0.8685889638065035*np.log(y1+2.51*y2/Re)
    y4=-0.8685889638065035*np.log(y1+2.51*y3/Re)
    f=y2-(y3-y2)*(y3-y2)/(y4-2*y3+y2)
    #
    return 1/(f*f)

name_[7]='Vatankhah_K_08'
@jit(parallel=True)
def f_7(Re,epsD): 
    y1=0.124*Re*epsD+np.log(0.4587*Re)
    f=0.8686*np.log(0.4587*Re/(np.exp(np.log(y1-0.31)*y1/(y1+0.9633))))
    return 1/(f*f)

name_[8]='zig1982'
@jit(parallel=True)
def f_8(Re,epsD): 
    y1=epsD/3.7-5.02/Re*np.log10(epsD/3.7-5.02/Re*np.log10(epsD/3.7+13/Re))
    f=-0.8685889638065035*np.log(y1)
    f=1/(f*f)
    f=np.log10(2.51/(Re*np.sqrt(f))+epsD/3.7)
    return 0.25/(f*f)

name_[9]='S.Alashkar_2012'
@jit(parallel=True)
def f_9(Re,epsD): 
    y1=epsD/3.7065
    y2=2.5226/Re
    f=np.log(y1-0.8686068432*y2*np.log(y1-0.8784893582*y2*np.log(y1+np.exp(np.log(1.665368035*y2)*0.8373492157))))
    return 1.325474505/(f*f)

name_[10]='Fang 2011'
@jit(parallel=True)
def f_10(Re,epsD): 
    y1=0.234*np.exp(np.log(epsD)*1.1007)
    y2=-60.525*np.exp(np.log(Re)*-1.1105)
    y3=56.291*np.exp(np.log(Re)*-1.0712)
    f=np.log(y1+y2+y3)
    return 1.613/(f*f)

name_[11]='Evangelides'
@jit(parallel=True)
def f_11(Re,epsD): 
    y1=7-np.log10(Re)
    y1=y1*y1*y1*y1
    y1=0.2479-0.0000947*y1
    y2=np.log10(epsD/3.615+7.366*np.exp(np.log(Re)*-0.9142))
    return y1/(y2*y2)

name_[12]='vatan19'
@jit(parallel=True)
def f_12(Re,epsD): 
    y1=0.8685889638065035 #2/np.log(10)  -a
    y2=0.39840637450199207 #1/2.51  -a1
    y3=0.12363404028060514  #   a2
    y4=y3*Re*epsD+np.log(Re/2.51) #s
    y5=1+0.49959/((y4+1)/np.log(y1*y4)-(1+5.448*y4)/(8*(y4+1))) #lambda
    f=y1*np.log(y2*Re)-y1*y4/(y4+y5)*np.log(y1*y4)
    return 1/(f*f)

name_[13]='newton2'
@jit(parallel=True)
def f_13(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re     #-2/log(10)*2,51
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2

name_[14]='Newton_CW_5iter'
@jit(parallel=True)
def f_14(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re     #-2/log(10)*2,51
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    z_=z_-(z_-np.log(y1+y2*z_))/(1-y2/(y1+y2*z_))
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2

name_[15]='Halley_CW_4iter'
@jit(parallel=True)
def f_15(Re,epsD): 
    z_=-6.880288946433447 #1/sqrt(0.028)*log(10)/(-2)
    y1=epsD/3.7
    y2=-2.180158299154324/Re #-2/log(10)*2,51
    f_=0
    temp=0
    fp_=0
    fpp_=0
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    f_=z_-np.log(y1+y2*z_)
    temp=y2/(y1+y2*z_)
    fp_=1-temp
    fpp_=-temp*temp
    z_=z_-2*f_*fp_/(2*fp_*fp_-f_*fpp_)
    #
    return 1.325474527619600/(z_*z_)  #(ln(10)/(-2))^2
    
name_[16]='Praks'
@jit(parallel=True)
def f_16(Re,epsD): 
    a=-0.8685889638065035#(2/np.log(10))
    delta_=74205.5+1000*epsD*Re
    A=8-a*np.log(16/Re+epsD/3.7)
    B=-74914381.46/(delta_*delta_)
    C=1391459721232.67/(delta_*delta_*delta_)
    z_=8-A-0.5*A*A*B
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    z_=a*np.log(2.51/Re*z_+epsD/3.7)
    #
    return 1/(z_*z_) #%(ln(10)/(-2))^2

In [18]:

def f_(Re,epsD,no_):
    if   no_==0:
        return f_0(Re,epsD)
    elif no_==1:
        return f_1(Re,epsD)
    elif no_==2:
        return f_2(Re,epsD)
    elif no_==3:
        return f_3(Re,epsD)
    elif no_==4:
        return f_4(Re,epsD)
    elif no_==5:
        return f_5(Re,epsD)
    elif no_==6:
        return f_6(Re,epsD)
    elif no_==7:
        return f_7(Re,epsD)
    elif no_==8:
        return f_8(Re,epsD)
    elif no_==9:
        return f_9(Re,epsD)
    elif no_==10:
        return f_10(Re,epsD)
    elif no_==11:
        return f_11(Re,epsD)
    elif no_==12:
        return f_12(Re,epsD)
    elif no_==13:
        return f_13(Re,epsD)
    elif no_==14:
        return f_14(Re,epsD)
    elif no_==15:
        return f_15(Re,epsD)
    elif no_==16:
        return f_16(Re,epsD)    

In [19]:
print('check accuracy with CW')
@njit(parallel=True)
def func_errCW(Re,epsD,z_):
    AA=1/np.sqrt(z_)+2/np.log(10)*np.log(epsD/3.7+2.51/(Re*np.sqrt(z_)))
    print('cw err=', np.max(np.abs(AA))*100,'%')

def func_err(z_,z_hat):
    return max(abs((z_-z_hat)/z_))*100

check accuracy with CW


In [20]:
z_=f_(Re,epsD,1)
func_errCW(Re,epsD,z_)

cw err= 5.329070518200751e-13 %


In [21]:
for i in range(NN+1):
    result = %timeit -o f_(Re,epsD,i)
    time_[i]=result.average
    MARE_[i]=func_err(z_,f_(Re,epsD,i))    


96.4 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.03 s ± 53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
154 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
174 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
97.6 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
187 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
111 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
124 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
155 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
147 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
174 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
128 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
107 ms ± 791 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
84.3 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
178 ms ± 5.48 ms per loop (mean ± std. dev. of 7 ru

In [22]:
print('python-numba-numpy-intel 3610qm-N=',N)

for i in range(NN+1):
    print('%20s'%name_[i],'\t,t=\t',round(time_[i],6),'\ts,t/t_SJ=\t',round(time_[i]/time_[0],3),'\t,err=\t',MARE_[i],'\t%')
    


python-numba-numpy-intel 3610qm-N= 10000000
         swamee jain 	,t=	 0.096353 	s,t/t_SJ=	 1.0 	,err=	 3.357650941912486 	%
                cw30 	,t=	 1.033273 	s,t/t_SJ=	 10.724 	,err=	 0.0 	%
                 cw5 	,t=	 0.154122 	s,t/t_SJ=	 1.6 	,err=	 0.012976867339158002 	%
       goudar-sonnad 	,t=	 0.17351 	s,t/t_SJ=	 1.801 	,err=	 1.0368606542558987e-10 	%
        haaland_1983 	,t=	 0.097561 	s,t/t_SJ=	 1.013 	,err=	 3.419004546424302 	%
            Barr1981 	,t=	 0.186806 	s,t/t_SJ=	 1.939 	,err=	 0.5336404816841809 	%
       serghides1984 	,t=	 0.110892 	s,t/t_SJ=	 1.151 	,err=	 0.003099141577008486 	%
      Vatankhah_K_08 	,t=	 0.124022 	s,t/t_SJ=	 1.287 	,err=	 0.0515427274021478 	%
             zig1982 	,t=	 0.154545 	s,t/t_SJ=	 1.604 	,err=	 0.014312920358080832 	%
     S.Alashkar_2012 	,t=	 0.147347 	s,t/t_SJ=	 1.529 	,err=	 0.11916005638955131 	%
           Fang 2011 	,t=	 0.173699 	s,t/t_SJ=	 1.803 	,err=	 0.49106149682993877 	%
         Evangelides 	,t=	 0.128431 	s,t/